In [1]:
import warnings
warnings.filterwarnings('ignore')

import random
import numpy as np
import pandas as pd
import tensorflow as tf

tf.logging.set_verbosity(tf.logging.ERROR)

# 1. Data Preparation
* Đọc dữ liệu

In [2]:
df = pd.read_csv("./data/songdata.csv")

In [3]:
df.head()

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \nAnd..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \nTouch me gentl..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \nWhy I had t...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [4]:
df.shape[0]

57650

* Số lượng nghệ sĩ

In [5]:
len(df['artist'].unique())

643

* Số bài hát mà từng nghệ sĩ có

In [6]:
df['artist'].value_counts()[:10]

Donna Summer        191
Gordon Lightfoot    189
Bob Dylan           188
George Strait       188
Cher                187
Loretta Lynn        187
Reba Mcentire       187
Alabama             187
Dean Martin         186
Chaka Khan          186
Name: artist, dtype: int64

* Số bài hát trung bình mà một nghệ sĩ có

In [7]:
df['artist'].value_counts().values.mean()

89.65785381026438

* Gôm tất cả bài hát lại với nhau

In [8]:
data = ", ".join(df['text'])

In [9]:
data[:400]

"Look at her face, it's a wonderful face  \nAnd it means something special to me  \nLook at the way that she smiles when she sees me  \nHow lucky can one fellow be?  \n  \nShe's just my kind of girl, she makes me feel fine  \nWho could ever believe that she could be mine?  \nShe's just my kind of girl, without her I'm blue  \nAnd if she ever leaves me what could I do, what could I do?  \n  \nAnd when we go f"

* Chúng ta đang build một model cấp kí tự bằng RNN, nên chúng ta cần lưu các unique characters trong dataset vào `chars`

In [10]:
chars = sorted(list(set(data)))

In [11]:
print(chars[:30])

['\n', ' ', '!', '"', "'", '(', ')', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H']


* Lấy size của `chars` và lưu vào biến `vocab_size`

In [12]:
vocab_size = len(chars)

In [13]:
vocab_size

76

* Vì neuron network chỉ nhận đầu vào là số, nên cần chuyển đổi tất cả các kí tự này thành số.
* Chúng ta ánh xạ tất cả các kí tự trong `chars` vào `char_to_ix` và `ix_to_char`.

In [14]:
char_to_ix = {ch:ix for ix, ch in enumerate(chars)} 
ix_to_char = {ix:ch for ix, ch in enumerate(chars)}

In [15]:
char_to_ix['s']

68

In [16]:
ix_to_char[68]

's'

* One hot encoded vector

In [17]:
def one_hot_encoder(index):
    return np.eye(vocab_size)[index]

In [18]:
one_hot_encoder(0)

array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0.])

# 2. Định nghĩa các network parameters

In [19]:
# xác định số node trong hidden layer
hidden_size = 100

# định nghĩa chiều dài của chuổi input và output
seq_length = 25

# định nghĩa learning rate
learning_rate = 1e-1

# seed
seed_value = 42
tf.set_random_seed(seed_value)
random.seed(seed_value)

# 3. Defining Placeholders
* Định nghĩa các placeholder cho input và output

In [20]:
inputs = tf.placeholder(shape=[None, vocab_size], dtype=tf.float32, name='inputs')
targets = tf.placeholder(shape=[None, vocab_size], dtype=tf.float32, name='targets')

* Định nghĩa các placeholder cho initial hidden layer

In [21]:
init_state = tf.placeholder(shape=[1, hidden_size], dtype=tf.float32, name='state')

* Định nghĩa initializer các weights của RNN

In [22]:
initializer = tf.random_normal_initializer(stddev=0.1)

# 4. Định nghĩa forward propagation
  $$h_t = \mathrm{tanh}(Ux_t + Wh_{t - 1} + bh)$$
  $$\widehat{y} = Vh_t + by$$

In [23]:
with tf.variable_scope("RNN") as scope:
    h_t = init_state
    y_hat = []
    
    for t, x_t in enumerate(tf.split(inputs, seq_length, axis=0)):
        if t > 0: scope.reuse_variables()
        
        # input to hidden layer weights
        U = tf.get_variable('U', [vocab_size, hidden_size], initializer=initializer)
        
        # hidden to hidden layer weights
        W = tf.get_variable('W', [hidden_size, hidden_size], initializer=initializer)
        
        # hidden to output layer weights
        V = tf.get_variable('V', [hidden_size, vocab_size], initializer=initializer)
        
        # bias for hidden layer
        bh = tf.get_variable('bh', [hidden_size], initializer=initializer)
        
        # bias for output layer
        by = tf.get_variable('by', [vocab_size], initializer=initializer)
        
        h_t = tf.tanh(tf.matmul(x_t, U) + tf.matmul(h_t, W) + bh)
        
        y_hat_t = tf.matmul(h_t, V) + by
        y_hat.append(y_hat_t) 

* Apply `softmax` on the output and get the probabilities

In [24]:
output_softmax = tf.nn.softmax(y_hat[-1])
outputs = tf.concat(y_hat, axis=0)

* Compute the cross-entropy loss

In [25]:
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=targets, logits=outputs))

* Store the final hidden state of the RNN in `hprev`. We use this final hidden state for making predictions

In [26]:
hprev = h_t

# 5. Defining BPTT
* Bây giờ, chúng ta sẽ thực thi BPTT bằng Adam optimizer. Chúng ta cũng sẽ thực thi gradient clipping to avoid the exploding gradients problem.

In [27]:
# 1. Initialize the Adam optimizer
minimizer = tf.train.AdamOptimizer()

# 2. Compute the gradients of the loss with the Adam optimizer
gradients = minimizer.compute_gradients(loss)

# 3. Set the threshold for the gradient clipping
threshold = tf.constant(5.0, name='grad_clipping')

# 4. Clip the gradients that exceed the threshold and bring it to the range
clipped_gradients = []
for grad, var in gradients:
    clipped_grad = tf.clip_by_value(grad, -threshold, threshold)
    clipped_gradients.append((clipped_grad, var))
    
# 5. Update the gradients with the clipped gradients
updated_gradients = minimizer.apply_gradients(clipped_gradients)

# 6. Start generating songs

In [28]:
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)